# NEU (Reconfigurations Map and Related Functions)

### Basic Algorithm (NEU-OLS)

1. Perform Basic Algorithm (in this case OLS)
2. Map predictions to their graph; ie $x\mapsto (x,\hat{f}_{OLS}(x))$ where $\hat{f}_{OLS}$ is the least-squares regression function.

## Initializations:

In [1]:
# Deep Learning & ML
import tensorflow as tf
import tensorflow_probability as tfp
import keras as K
from keras import backend as Kback
from keras.models import Sequential
from keras import layers
from keras import utils as np_utils
from scipy import linalg as scila

# Linear Regression
from sklearn.linear_model import LinearRegression

# General
import numpy as np
import time

# Alerts
import os as beepsnd

# General Outputs
print('TensorFlow:', tf.__version__)

Using TensorFlow backend.


TensorFlow: 2.1.0


#### Prepare data for NEU

In [2]:
# Reshape Data Into Compatible Shape
data_x = np.array(data_x).reshape(-1,d)
data_y = np.array(data_y)
# Perform OLS Regression
linear_model = LinearRegression()
reg = linear_model.fit(data_x, data_y)
model_pred_y = linear_model.predict(data_x)
# Map to Graph
data_NEU = np.concatenate((data_x,model_pred_y.reshape(-1,D)),1)
NEU_targets  = data_y.reshape(-1,D)

NameError: name 'data_x' is not defined

## Helper Functions:

In [ ]:
#----------------------------#
# Helper Functions T/F Functions
#----------------------------#
def bump_True(x):
    x_out = 1- tf.math.pow(tf.math.abs(x),2)
    x_out = tf.math.exp(-tf.math.divide(1,x_out))
    return x_out

def bump_False(x):
    x_out = 0
    return x_out

# Approximation to max(0,x) function by smooth function
# https://math.stackexchange.com/questions/517482/approximating-a-maximum-function-by-a-differentiable-function
def abs_helper(x):
    return tf.math.sqrt(tf.math.pow(x,2)+(10**(-5)))

def max_helper(x):
    return tf.math.multiply(abs_helper(x) + x,.5)

def soft_indicator(x):
    return max_helper(tf.math.sign(x))

def bump_function(x):
    # Gaussian rescaled to [-1,1]^d
    bump_out = 1-tf.math.pow(x,2)        
    bump_out = tf.math.divide(-1,bump_out)
    bump_out = tf.math.exp(bump_out)
    # Indicator
    indicator = soft_indicator(1-x)
    # 0 outside of [-1,1]^d
    bump_out = tf.math.multiply(bump_out,indicator)
    return bump_out

### Build Reconfiguration Unit
$$
x \mapsto \exp\left(
\psi(x-c;\sigma) X
\right) (x-c) + c
$$
where:
#### Workflow
1. Shifts $x \in \mathbb{R}^d$ to $x- c$; c trainable.
2. Applies the map $\psi(x;\sigma)\triangleq e^{\frac{\sigma}{\sigma-|x|}}I_{\{|x|<\sigma\}}$ component-wise.  
3. Applies transformation $x \mapsto x +b$, $b \in \mathbb{R}^d$ trainable.
4. Applies the diagonalization map to that output: $ \left(x_1,\dots,x_d\right)\mapsto
                \begin{pmatrix}
                x_1 & & 0\\
                &\ddots &\\
                0 & & x_d\\
                \end{pmatrix}.$
5. Applies map $X \mapsto XA$, $A$ is a trainable $d\times d$ matrix.
6. Applies matrix exponential.
7. Multiplies output with result of (1).
8. Re-centers output to $x +c$ where $c$ is as in (1).

### Helper Function: Build and Training NEU Units (Core)

In [3]:
class Reconfiguration_unit_steps(tf.keras.layers.Layer):

    def __init__(self, *args, **kwargs):
        super(Reconfiguration_unit_steps, self).__init__(*args, **kwargs)

    def build(self, input_shape):
        self.location = self.add_weight(name='location',
                                     shape=input_shape[1:],
                                     initializer='GlorotUniform',
                                     trainable=True)
        
        
# #     Step 3
#         self.scale = self.add_weight(name='location',
#                                     shape=input_shape[1:],
#                                     initializer='GlorotNormal',
#                                     trainable=True)
    
#         self.threshold = self.add_weight(name='threshold',
#                             shape=input_shape[1:],
#                             initializer='GlorotUniform',
#                             trainable=True)
        

        # STEP 3 - NEW
        # INITIALIZE KERNELS
        self.kernel_a = self.add_weight(name='kernel_a ',
                                    shape=input_shape[1:],
                                    initializer='GlorotUniform',
                                    trainable=True)

        #inverse width
        self.kernel_b = self.add_weight(name='kernel_b',
                                    shape=input_shape[1:],
                                    initializer='ones',
                                    trainable=True)

        #center
        self.kernel_c = self.add_weight(name='kernel_c',
                                    shape=input_shape[1:],
                                    initializer='zeros',
                                    trainable=True)
 

    
        ## Steps 4-9
        self.tangentbiases = self.add_weight(name='tangentbiases',
                                    shape=input_shape[1:],
                                    initializer='GlorotUniform',
                                    trainable=True)
        self.tangentweights = self.add_weight(name='tangentweights',
                                    shape=input_shape[1:],
                                    initializer='GlorotUniform',
                                    trainable=True)
        self.location_b = self.add_weight(name='location_b',
                                    shape=input_shape[1:],
                                    initializer='GlorotUniform',
                                    trainable=True)
    
    
    def call(self, input):
        ### Steps 1-2
        # Relocated
        output_steps_1_2 = input + self.location
        
        ### Step 3
        # Old
#         thresholdin_3 = tf.math.multiply(tf.math.pow(self.scale,2),output_steps_1_2)
#         thresholdin_3 = output_steps_1_2-self.threshold
#         thresholdin_3 = bump_function(thresholdin_3)    
        # NEW
        exp_arg = - self.kernel_b * tf.math.square(output_steps_1_2 - self.kernel_c)
        output_step_3 = self.kernel_a * tf.math.exp(exp_arg)

        
#         output_step_3 = tf.math.multiply(output_step_3,thresholdin_3)
        
        ### Step 4-9
        x_out = tf.math.multiply(tf.math.abs(self.tangentweights),output_step_3) + self.tangentbiases
        
        # 7. Apply Matrix Exponential
        x_out = tf.linalg.diag(x_out)
        #------------------------------#
        # Using an approximation of the matrix exponential (tf.linalg.expm(x_out)) is prefeable 
        # **Since:** TF uses Sylvester's method to for (fast) computation but this requires additional assumptions on the matrix which are typically not satified...
        # **Instead:** Use a truncated power series representation (standard definition of expm) of order 4
        x_out = tf.linalg.diag(tf.ones(d+D)) + x_out + tf.linalg.matmul(x_out,x_out)/2 + tf.linalg.matmul(x_out,tf.linalg.matmul(x_out,x_out))/6 +tf.linalg.matmul(x_out,tf.linalg.matmul(x_out,tf.linalg.matmul(x_out,x_out)))/24 #+tf.linalg.matmul(x_out,tf.linalg.matmul(x_out,tf.linalg.matmul(x_out,tf.linalg.matmul(x_out,x_out))))/120
        #x_out = tf.linalg.expm(x_out)
        
        # 8. Muliply by output of (1)
        x_out = tf.linalg.matvec(x_out,input)
        
        # 9. Recenter Transformed Data
        x_out = x_out + self.location_b
        
        # Return Output
        return x_out

### Helper Function: Projection Layer (For Regression)
Maps $\mathbb{X}\left((x,f(x))\mid \theta \right) \in \mathbb{R}^{d\times D}$ to an element of $\mathbb{R}^D$ by post-composing with the second canonical projection
$$
(x_1,x_2)\mapsto x_2
,
$$
where $x_1 \in \mathbb{R}^d$ and $x_2 \in \mathbb{R}^D$.  

In [4]:
projection_layer = tf.keras.layers.Lambda(lambda x: x[:, -D:])

### Helper Functions: Compiling and Training NEU-OLS

#### First Unit
These are helper functions for training the reconfiguration map.

Build and greedily-initialize the first reconfiguration unit.

In [5]:
# define and fit the base model
def get_base_model(trainx, trainy, Pre_Epochs_in):
    # Define Model
    #----------------#
    # Initialize
    input_layer = tf.keras.Input(shape=[d+D])
    # Apply Reconfiguration Unit
    reconfigure  = Reconfiguration_unit_steps()
    current_layer = reconfigure(input_layer)
    # Output
    output_layer = projection_layer(current_layer)
    reconfiguration_basic = tf.keras.Model(inputs=[input_layer], outputs=[output_layer])
    
    # Compile Model
    #----------------#
    # Define Optimizer
    optimizer_on = tf.keras.optimizers.SGD(learning_rate=10**(-2), momentum=0.01, nesterov=True)
    # Compile
    reconfiguration_basic.compile(loss = 'mse',
                    optimizer = optimizer_on,
                    metrics = ['mse'])
    
    # Fit Model
    #----------------#
    reconfiguration_basic.fit(trainx, trainy, epochs=Pre_Epochs_in, verbose=0)
        
    # Return Output
    return reconfiguration_basic

#### Greedy Initialization of Subsequent Units
Build reconfiguration and pre-train using greedy approach.

In [6]:
def add_reconfiguration_unit_greedily(model, trainx, trainy, Pre_Epochs_in):

    # Dissasemble Network
    layers = [l for l in model.layers]

    # Define new reconfiguration unit to be added
    new_reconfiguration_unit  = Reconfiguration_unit_steps()
    current_layer_new = new_reconfiguration_unit(layers[len(layers)-2].output)

    # Output Layer
    output_layer_new = projection_layer(current_layer_new)

    for i in range(len(layers)):
        layers[i].trainable = False


    # build model
    new_model = tf.keras.Model(inputs=[layers[0].input], outputs=output_layer_new)
    #new_model.summary()


    # Compile new Model
    #-------------------#
    # Define Optimizer
    optimizer_on = tf.keras.optimizers.SGD(learning_rate=10**(-2), momentum=0.01, nesterov=True)
    # Compile Model
    new_model.compile(loss = 'mse',
                    optimizer = optimizer_on,
                    metrics = ['mse'])

    # Fit Model
    #----------------#
    new_model.fit(trainx, trainy, epochs=Pre_Epochs_in, verbose=0)

    # Return Output
    return new_model

#### Train and Compile (entire) reconfiguration using greedy-initializations past from previous helper functions.
Train reconfiguration together (initialized by greedy) layer-wise initializations.

In [7]:
def build_reconfiguration(model_greedy_initialized, trainx, trainy, Full_Epochs_in):

    # Dissasemble Network
    layers = [l for l in model_greedy_initialized.layers]

    # Define new reconfiguration unit to be added
    new_reconfiguration_unit  = Reconfiguration_unit_steps()
    current_layer_new = new_reconfiguration_unit(layers[len(layers)-2].output)

    # Output Layer
    output_layer_new = projection_layer(current_layer_new)

    for i in range(len(layers)):
        layers[i].trainable = True


    # build model
    reconfiguration = tf.keras.Model(inputs=[layers[0].input], outputs=output_layer_new)
    #new_model.summary()



    # Compile new Model
    #-------------------#
    # Define Optimizer
    optimizer_on = tf.keras.optimizers.SGD(learning_rate=10**(-2), momentum=0.01, nesterov=True)
    # Compile Model
    reconfiguration.compile(loss = 'mse',
                    optimizer = optimizer_on,
                    metrics = ['mse'])

    # Fit Model
    #----------------#
    reconfiguration.fit(trainx, trainy, epochs=Full_Epochs_in, verbose=1)

    # Return Output
    return reconfiguration

## Train NEU-OLS

In [8]:
# Base Model
model = get_base_model(data_NEU,NEU_targets,Pre_Epochs)

# Greedy Initialization
NEU_OLS_Greedy_init = get_base_model(data_NEU,NEU_targets,Pre_Epochs)
for i in range(N_Reconfigurations):
    NEU_OLS_Greedy_init = add_reconfiguration_unit_greedily(NEU_OLS_Greedy_init,data_NEU,NEU_targets,Pre_Epochs)
    # Update User on Status of Initialization
    print((i/N_Reconfigurations))
    
# Train Full Model Using Initializatoins
NEU_OLS = build_reconfiguration(NEU_OLS_Greedy_init,data_NEU,NEU_targets,Full_Epochs)



# Predictions (for comparison: TEMP)
NEU_OLS_single_unit_prediction = model.predict(data_NEU)
NEU_OLS_greedy_initializations = NEU_OLS_Greedy_init.predict(data_NEU)
NEU_OLS_prediction = NEU_OLS(data_NEU)

NameError: name 'data_NEU' is not defined

In [9]:
# import matplotlib.pyplot as plt
# Adjust Figure Details
plt.figure(num=None, figsize=(12, 10), dpi=80, facecolor='w', edgecolor='k')

# Plot Models
plt.plot(data_x,true_y,color='k',label='true',linestyle='--')
plt.plot(data_x,NEU_OLS_single_unit_prediction,color='b',label='NEU-OLS')
plt.plot(data_x,NEU_OLS_greedy_initializations,color='g',label='NEU-OLS')
plt.plot(data_x,NEU_OLS_prediction,color='Aqua',label='NEU-OLS')
plt.plot(data_x,model_pred_y,color='r',label='NEU-OLS')

NameError: name 'plt' is not defined

HERE!!!
https://machinelearningmastery.com/greedy-layer-wise-pretraining-tutorial/